In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mibitrans.data.parameters import AttenuationParameters
from mibitrans.data.parameters import HydrologicalParameters
from mibitrans.data.parameters import ModelParameters
from mibitrans.data.parameters import SourceParameters
from mibitrans.transport.models import Anatrans
from mibitrans.visualize.plot_line import centerline
from scipy.special import erfcx, erf, erfc
import mibitrans as mbt

In [ ]:
ft = 3.281
hydro = HydrologicalParameters(
    velocity=350/ft/365,
    h_gradient=0.048,        # Hydraulic gradient [-]
    h_conductivity=0.495,    # Hydraulic conductivity [m/day]
    porosity=0.25,           # Effective soil porosity [-]
    alpha_x=13.3/ft,         # Longitudinal dispersivity, in [m]
    alpha_y=1.3/ft,          # Transverse horizontal dispersivity, in [m]
    alpha_z=0                # Transverse vertical dispersivity, in [m]
)

att = AttenuationParameters(
    # Soil bulk density in [g/m^3]
    bulk_density=1.7,
    # Partition coefficient of the transported contaminant to soil organic matter, in [m^3/g]
    partition_coefficient=38,
    # Fraction of organic material in the soil [-]
    fraction_organic_carbon=5.7e-5,
    # Molecular diffusion, in [m2/day]
    diffusion=0,
    # Contaminant half life, in [days]
    half_life=0
)

source = SourceParameters(
    source_zone_boundary=np.array([30/ft, 35/ft, 40/ft]),
    source_zone_concentration=np.array([13.68, 2.508, 0.057]),
    depth=10/ft,
    total_mass="inf"
)

model = ModelParameters(
    # Model extent in the longitudinal (x) direction in [m].
    model_length = 1000/ft,
    # Model extent in the transverse horizontal (y) direction in [m].
    model_width = 100/ft,
    # Model duration in [days].
    model_time = 10 * 365,
    # Model grid discretization step size in the longitudinal (x) direction, in [m].
    dx = 1/ft,
    # Model grid discretization step size in the transverse horizontal (y) direction, in [m].
    dy = 1/ft,
    # Model time discretization step size, in [days]
    dt = 365 / 20
)

In [ ]:
class AnatransAlternativeSource(Anatrans):
    """Mibitrans model class with alternative calculations."""

    def __init__(
    self,
    hydrological_parameters,
    attenuation_parameters,
    source_parameters,
    model_parameters,
    verbose=False,
    ):
        """Initialize model class."""
        super().__init__(hydrological_parameters, attenuation_parameters, source_parameters, model_parameters, verbose)

    def _equation_term_source_decay(self, xxx, ttt):
        #adapted source term
        term = np.exp(-self.k_source * ttt)
        # Term can be max 1; can not have 'generation' of solute ahead of advection.
        return np.where(term > 1, 1, term)

    def _calculate_concentration_for_all_xyt(self, xxx, yyy, ttt):
        cxyt = 0
        #adapted decay_sqrt
        decay_sqrt = np.sqrt(1 + 4 * (self._decay_rate - self.k_source) * self._hyd_pars.alpha_x / self.rv)
        x_term = self._equation_term_x(xxx, ttt, decay_sqrt)
        additional_x = self._equation_term_additional_x(xxx, ttt, decay_sqrt)
        z_term = self._equation_term_z(xxx)
        source_decay = self._equation_term_source_decay(xxx, ttt)
        for i in range(len(self.c_source)):
            y_term = self._equation_term_y(i, xxx, yyy)
            cxyt_step = 1 / 8 * self.c_source[i] * source_decay * (x_term + additional_x) * y_term * z_term
            cxyt += cxyt_step
        if self._mode == "instant_reaction":
            self.cxyt_noBC = cxyt.copy()
            cxyt -= self.biodegradation_capacity
            cxyt = np.where(cxyt < 0, 0, cxyt)
        self.has_run = True
        return cxyt

In [ ]:
ana_object = Anatrans(hydro, att, source, model)
ana_results = ana_object.run()

alt_object = AnatransAlternativeSource(hydro, att, source, model)
alt_results = alt_object.run()

In [ ]:
%matplotlib ipympl

In [ ]:
plt.clf()
ana_results.centerline(color="green")
alt_results.centerline(color="red", linestyle="--")
plt.show()

In [ ]:
plt.clf()
anim = centerline(model=[ana_results, alt_results],
                  legend_names=["Anatrans model", "Alternative model"],
                  linestyle=":",
                  animate=True)
plt.show()

In [ ]:
class AnatransAlternativeDispersion(Anatrans):
    """Mibitrans model class with alternative calculations."""

    def __init__(
    self,
    hydrological_parameters,
    attenuation_parameters,
    source_parameters,
    model_parameters,
    verbose=False,
    ):
        """Initialize model class."""
        super().__init__(hydrological_parameters, attenuation_parameters, source_parameters, model_parameters, verbose)


    def _equation_term_z(self, xxx, ttt):
        inner_term = self._src_pars.depth / (2 * np.sqrt(self._hyd_pars.alpha_z * self.rv * ttt))
        return erf(inner_term) - erf(-inner_term)

    def _equation_term_y(self, i, xxx, yyy, ttt):
        div_term = 2 * np.sqrt(self._hyd_pars.alpha_y * self.rv * ttt)
        term = erf((yyy + self.y_source[i]) / div_term) - erf((yyy - self.y_source[i]) / div_term)
        term[np.isnan(term)] = 0
        return term

    def _calculate_concentration_for_all_xyt(self, xxx, yyy, ttt):
        cxyt = 0
        #adapted decay_sqrt
        decay_sqrt = np.sqrt(1 + 4 * (self._decay_rate - self.k_source) * self._hyd_pars.alpha_x / self.rv)
        x_term = self._equation_term_x(xxx, ttt, decay_sqrt)
        additional_x = self._equation_term_additional_x(xxx, ttt, decay_sqrt)
        z_term = self._equation_term_z(xxx, ttt)
        source_decay = self._equation_term_source_decay(xxx, ttt)
        for i in range(len(self.c_source)):
            y_term = self._equation_term_y(i, xxx, yyy,ttt)
            cxyt_step = 1 / 8 * self.c_source[i] * source_decay * (x_term + additional_x) * y_term * z_term
            cxyt += cxyt_step
        if self._mode == "instant_reaction":
            self.cxyt_noBC = cxyt.copy()
            cxyt -= self.biodegradation_capacity
            cxyt = np.where(cxyt < 0, 0, cxyt)
        self.has_run = True
        return cxyt

In [ ]:
plt.clf()
alt_disp_object = AnatransAlternativeDispersion(hydro, att, source, model)
alt_disp_results = alt_disp_object.run()
ana_results.centerline(color="green")
alt_results.centerline(color="red", linestyle="--")
alt_disp_results.centerline(color="blue", linestyle="-.")
plt.show()

In [ ]:
plt.clf()
anim = centerline(model=[ana_results, alt_results, alt_disp_results],
                  legend_names=["Anatrans model", "Alternative model", "Alt disp model"],
                  linestyle=":",
                  animate=True)
plt.show()

In [ ]:
plt.clf()
anim = mbt.plume_3d(model=alt_disp_results,
                    animate=True,
                    cmap='viridis')
plt.show()

In [ ]:

anim = ana_results.plume_3d(
                    animate=True,
                    cmap='viridis')
plt.show()